In [0]:
%sql
with tbl1 as (
select date_ym
        , context_name, message_id
        , X_Device_Country
        , table_nm
        , X_Device_Product
        , if(X_Device_SDK_VERSION like '1%', X_Device_SDK_VERSION, concat('0', X_Device_SDK_VERSION)) as X_Device_SDK_VERSION
        , X_Device_Platform
        , sum(UD) as UD
from sandbox.z_sam351_kim.nl_vsc_dpv_ud_mm
where 1=1
    and UD>1000
    and date_ym between '2025-11' and '2026-01'
    and substr(X_Device_Platform, 1, 1) IN ('W', 'C')
group by date_ym
        , context_name, message_id
        , X_Device_Country
        , table_nm
        , X_Device_Product
        , X_Device_SDK_VERSION
        , X_Device_Platform
)
select 'eic' as ric_nm
        , context_name
        , message_id
        , date_ym
        , X_Device_Country as country_code
        , table_nm
        , X_Device_Product as platform_version
        , X_Device_Platform as platform_code
        , X_Device_SDK_VERSION as device_platform_version
        , UD
        , round(UD / sum(UD) over (partition by X_Device_Country, table_nm), 4) as UD_ratio
        , round(sum(UD) over (partition by X_Device_Country, table_nm order by X_Device_SDK_VERSION) / sum(UD) over (partition by X_Device_Country, table_nm), 4) as UD_ratio_accum
        , sum(UD) over (partition by X_Device_Country, table_nm) as UD_tot
from tbl1
order by date_ym, country_code, table_nm, platform_version, platform_code, device_platform_version